# Introduction/Business Problem

In this project we are going to finding a good location in New york for my relocation and this report targets all the families planning to relocate the New york.

Since there are lots of communities in New york  we will **find a good community in the neighborhood** and at the same time with **good school ratings.** We would also prefer **locations as close to city as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

# Data

Based on the definition of our problem ,factors that will influence our decision are:

* number of existing schools available in the neighborhood
* distance to schools in the neighborhood, if any
* distance of neighborhood from the city

**Following data sources will be needed to extract/generate the required information:**

* New York Neighborhood data will be gathered from the below website in a Json format:**https://geo.nyu.edu/catalog/nyu_2451_34572**
* **FourSquare API** will be used to explore schools near each New York neighborhood. This data needs to analyzed and find the frequency of schools available in each Neighborhood : **https://api.foursquare.com/v2/venues/explore?**
* Geopy library to get the latitude and longitude values of New York City Folium visualization library to display the data in the map, each circle mark to reveal the name of the neighborhood and its respective borough, for illustration purposes, simplified the map and segment and cluster only the neighborhoods. Machine learnings: K-means Clustering algorithm is used in this project. The data are clustered based on the similarity. There are 5 cluster used in this project to cluster the data.

In [1]:
import numpy as np 

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.6.4                |           py36_0         877 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         961 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0

The following packages will be UPDATED:

  conda                                        4.6.3-py36_0 --> 4.6.4-py36_0
  geopy              conda-forge/linux-64::geopy-1.11.0-py~ --> conda-forge/noarch::geopy-1.18.1-py_0



geopy-1.18.1         

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
#newyork_data['features']

neighborhoods_data= newyork_data['features']

neighborhoods_data[2]

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Data downloaded!


In [4]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [5]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

The geograpical coordinate of New York City are 40.7308619, -73.9871558.


In [6]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [7]:
CLIENT_ID = 'ZOILLFGKUANWRS1XLNZBICTQA3DOTLGAYAWZPDVIRKEFF0SU' # your Foursquare ID
CLIENT_SECRET = 'K5I03UN4BRRQQJNKANTPKCIVXBFYHD3HQIOGOIKSISXKLASY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZOILLFGKUANWRS1XLNZBICTQA3DOTLGAYAWZPDVIRKEFF0SU
CLIENT_SECRET:K5I03UN4BRRQQJNKANTPKCIVXBFYHD3HQIOGOIKSISXKLASY


In [8]:
manhattan_data.loc[0, 'Neighborhood']
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [9]:
radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query=school&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ZOILLFGKUANWRS1XLNZBICTQA3DOTLGAYAWZPDVIRKEFF0SU&client_secret=K5I03UN4BRRQQJNKANTPKCIVXBFYHD3HQIOGOIKSISXKLASY&v=20180605&ll=40.87655077879964,-73.91065965862981&query=school&radius=500&limit=100'

In [10]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c6c6d69351e3d13a6499801'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'query': 'school',
  'totalResults': 17,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4de6ee22152016a6c0757bf4',
       'name': 'The New Visions Charter High School for the Humanities',
       'location': {'address': '99 Terrace View Ave',
        'lat': 40.877391667555415,
        'lng': -73.91276883456942,


In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)


# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues

,name,categories,lat,lng
0,The New Visions Charter High School for the Hu...,School,40.877392,-73.912769
1,P.S 207,School,40.878314,-73.905977
2,John F. Kennedy High School,High School,40.876944,-73.912976
3,Bronx Theatre High School,High School,40.875952,-73.912384
4,marble hill high school,High School,40.877414,-73.912597
5,John F. Kennedy High School,High School,40.877266,-73.912746
6,Korean School Of New York,Language School,40.878851,-73.912671
7,Public school 207,School,40.878345,-73.906568
8,ELLIS Preparatory Academy,High School,40.875809,-73.912597
9,International Leadership Charter School,High School,40.880955,-73.909562


In [15]:
print('There are {} uniques categories.'.format(len(nearby_venues['categories'].unique())))

There are 6 uniques categories.


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query=school&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [18]:
print(manhattan_venues.shape)
manhattan_venues.head()

(1080, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,The New Visions Charter High School for the Hu...,40.877392,-73.912769,School
1,Marble Hill,40.876551,-73.91066,P.S 207,40.878314,-73.905977,School
2,Marble Hill,40.876551,-73.91066,John F. Kennedy High School,40.876944,-73.912976,High School
3,Marble Hill,40.876551,-73.91066,Bronx Theatre High School,40.875952,-73.912384,High School
4,Marble Hill,40.876551,-73.91066,marble hill high school,40.877414,-73.912597,High School


In [19]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,10,10,10,10,10,10
Carnegie Hill,43,43,43,43,43,43
Central Harlem,17,17,17,17,17,17
Chelsea,23,23,23,23,23,23
Chinatown,50,50,50,50,50,50
Civic Center,33,33,33,33,33,33
Clinton,16,16,16,16,16,16
East Harlem,36,36,36,36,36,36
East Village,23,23,23,23,23,23


In [20]:
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]
manhattan_onehot.head()
manhattan_onehot.shape

(1080, 58)

In [21]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped
manhattan_grouped.shape

(40, 58)

In [22]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                       venue  freq
0                     School   0.4
1             Nursery School   0.1
2  College Academic Building   0.1
3               Music School   0.1
4            College Library   0.1


----Carnegie Hill----
               venue  freq
0             School  0.35
1        High School  0.12
2       Music School  0.12
3    Language School  0.09
4  Elementary School  0.09


----Central Harlem----
                       venue  freq
0                     School  0.47
1          Elementary School  0.12
2                     Church  0.12
3  College Academic Building  0.12
4              Middle School  0.12


----Chelsea----
               venue  freq
0        High School  0.48
1             School  0.22
2  Elementary School  0.09
3        Swim School  0.09
4        College Lab  0.04


----Chinatown----
               venue  freq
0             School  0.40
1        High School  0.16
2     Driving School  0.10
3       Music School  0.10
4  Eleme

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,School,Cooking School,College Library,Nursery School,College Academic Building,Preschool,Music School,College Residence Hall,College Auditorium,College Science Building
1,Carnegie Hill,School,Music School,High School,Language School,Elementary School,Driving School,Preschool,Middle School,Church,College Academic Building
2,Central Harlem,School,Elementary School,Middle School,College Academic Building,Church,Driving School,Arts & Crafts Store,Building,Dance Studio,Cooking School
3,Chelsea,High School,School,Elementary School,Swim School,Office,College Lab,Preschool,College Classroom,College Engineering Building,College Gym
4,Chinatown,School,High School,Music School,Driving School,Middle School,Elementary School,College Arts Building,College Classroom,Preschool,Church


In [25]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 2, 0, 0, 0, 3, 1, 0], dtype=int32)

In [26]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 2, 0, 0, 0, 3, 1, 0], dtype=int32)

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2,High School,School,Elementary School,Adult Education Center,Language School,Middle School,College Bookstore,Community College,College Theater,College Science Building
1,Manhattan,Chinatown,40.715618,-73.994279,0,School,High School,Music School,Driving School,Middle School,Elementary School,College Arts Building,College Classroom,Preschool,Church
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Music School,School,Elementary School,Church,Driving School,Art Gallery,College Classroom,Dance Studio,Cooking School,Community College
3,Manhattan,Inwood,40.867684,-73.921210,0,School,Driving School,College Academic Building,High School,Adult Education Center,Office,College Auditorium,Community College,Cooking School,College Theater
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,School,Elementary School,Driving School,University,College Science Building,High School,College Engineering Building,Middle School,Dance Studio,College Classroom


In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [29]:

manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,School,High School,Music School,Driving School,Middle School,Elementary School,College Arts Building,College Classroom,Preschool,Church
3,Inwood,School,Driving School,College Academic Building,High School,Adult Education Center,Office,College Auditorium,Community College,Cooking School,College Theater
12,Upper West Side,School,Music School,High School,Elementary School,Preschool,Church,Nursery School,Middle School,Dance Studio,Office
14,Clinton,School,Music School,Church,General College & University,High School,Middle School,Office,Playground,Elementary School,Adult Education Center
15,Midtown,School,Language School,Trade School,High School,Office,General College & University,Adult Education Center,Music School,Preschool,College Academic Building
16,Murray Hill,School,Language School,High School,Music School,Nursery School,Preschool,Daycare,Driving School,Office,Trade School
18,Greenwich Village,School,High School,Language School,Adult Education Center,College Academic Building,Nursery School,Law School,Office,General College & University,Music School
25,Manhattan Valley,School,Preschool,Elementary School,High School,College Academic Building,Nursery School,Church,Pool,Music School,Swim School
26,Morningside Heights,School,University,Elementary School,Church,Student Center,College Academic Building,Music School,Law School,College Administrative Building,College Library
28,Battery Park City,School,Cooking School,College Library,Nursery School,College Academic Building,Preschool,Music School,College Residence Hall,College Auditorium,College Science Building
